In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
from scipy.fft import fft,ifft,fftshift,fftfreq
import matplotlib as mpl
import numpy as np
from scipy.io import loadmat
import copy


In [3]:
# 1.---------------FUNCTIONS FOR COMPUTING MINIMUM DETECTABLE LAKE SIZE---------
def get_Dj(lamda,beta_nd,w_ft,k):
    # function for computing displacements D1 (in-phase with base) and D2 (anti-phase with base)
    g = beta_nd/k

    # relaxation function
    R1 =  (1/k)*((1+g)*np.exp(4*k) - (2+4*g*k)*np.exp(2*k) + 1 -g)
    D = (1+g)*np.exp(4*k) + (2*g+4*k+4*g*(k**2))*np.exp(2*k) -1 + g
    R = R1/D

    # transfer function
    T1 = 2*(1+g)*(k+1)*np.exp(3*k) + 2*(1-g)*(k-1)*np.exp(k)
    T = T1/D

    G1 = T*w_ft
    G2 = 1 + (lamda*R)**2

    # displacements
    D1 = ifft(G1/G2).real
    D2 = ifft(lamda*R*G1/G2).real

    return D1,D2

In [4]:
def get_Tj(D1,D2,x):
    # Times where elevation anomaly is maximized (T1) and minimized (T2)
    i0 = np.argmin(np.abs(x))

    T1 = np.pi - np.arctan(D2[i0]/D1[i0])
    T2 = 2*np.pi - np.arctan(D2[i0]/D1[i0])

    return T1,T2

In [5]:
def get_kappaj(T1,T2):
    # weights on the displacements:
    # kappa1 is the in-phase component
    # kappa2 is the anti-phase component
    kappa1 = np.cos(T2) - np.cos(T1)
    kappa2 = np.sin(T1) - np.sin(T2)

    return kappa1,kappa2

In [6]:
def get_Lh(Ls,beta_d,H):
    # compute the estimated length of the lake (Lh) given the true lake length (Ls),
    # dimensional friction (beta_d), and ice thickness (H)

    # discretization in frequency domain
    N = 2000
    x = np.linspace(-200,200,num=N)
    d = np.abs(x[1]-x[0])
    k = fftfreq(N,d)   # frequency
    k[0] = 1e-10       # set zero frequency to small number due to (integrable) singularity
    k *= 2*np.pi       # convert to SciPy's Fourier transform definition from version (angular
                       # freq. definition) used in model derivation

    w = w_base(x,Ls)                       # compute basal velocity anomaly

    w_ft = fft(w)                          # fourier transform (for numerical method)

    beta_nd = beta_d*H/(2*eta)             # non-dimensional friction parameter
                                           # relative to viscosity/ice thickness

    tr =  (4*np.pi*eta)/(rho*g*H)          # relaxation time
    lamda = t_pd/tr                        # ratio of oscillation time to relaxation time


    D1,D2 = get_Dj(lamda,beta_nd,w_ft,k)   # compute surface displacements

    T1,T2 = get_Tj(D1,D2,x)                # compute estimated highstand/lowstand times

    kappa1,kappa2 = get_kappaj(T1,T2)      # compute weights for displacements

    dH = kappa1*D1 + kappa2*D2             # compute surface elevation change anomaly

    # compute estimated lake length
    if np.size(x[np.abs(dH)>delta])>0:
        x0 = x[np.abs(dH)>delta]
    else:
        x0 = 0*x

    Lh = 2*np.max(x0)                 # (problem is symmetric with respect to x)

    return Lh

In [7]:
def get_min_Ls(Ls,beta_d,H):
    # compute the minimum detectable lake size by starting with a huge lake size
    # and gradually decreasing it until the estimated lake length vanishes

    Ls_min = Ls[-1]                 # initialize as smallest lake size in case
                                    # all lake sizes are detectable

    for i in range(np.size(Ls)):
        Lh = get_Lh(Ls[i],beta_d,H)

        if Lh < 1e-5:
            Ls_min = Ls[i]
            break

    return Ls_min

In [ ]:
# function for spatial component of basal vertical velocity anomaly
# default is a Gaussian
def w_base(x,Ls):
    sigma = Ls/6                        # define standard deviation for Gaussian
    w = np.exp(-0.5*(x/sigma)**2)
    return w
